# Домашнее задание 1 [10 баллов] 
# До 1.12.17 23:59

Задание выполняется в группе (1-4 человека). В случае использования какого-либо строннего источника информации обязательно дайте на него ссылку (поскольку другие тоже могут на него наткнуться). Плагиат наказывается нулём баллов за задание и предвзятым отношением в будущем.

Не все части обязательны для выполнения, однако вы можете быть дополнительно оштрафованы за небрежное за выполнение одной или двух частей вместо четырех.

При возниконовении проблем с выполнением задания обращайтесь с вопросами к преподавателю. Поэтому настоятельно рекомендуется выполнять задание заранее, оставив запас времени на всевозможные технические проблемы. Если вы начали читать условие в последний вечер и не успели из-за проблем с установкой какой-либо библиотеки — это ваши проблемы.


Результат выполнения задания — это отчёт в формате html на основе Jupyter Notebook. Нормальный отчёт должен включать в себя:
* Краткую постановку задачи и формулировку задания
* Описание **минимума** необходимой теории и/или описание используемых инструментов - не стоит переписывать лекции или Википедию
* Подробный пошаговый рассказ о проделанной работе
* Аккуратно оформленные результаты
* **Внятные выводы** – не стоит относится к домашнему заданию как к последовательности сугубо технических шагов, а стоит относится скорее как к небольшому практическому исследованию, у которого есть своя цель и свое назначение.

Небрежное его оформление отчета существенно отразится на итоговой оценке. Весь код из отчёта должен быть воспроизводимым, если для этого нужны какие-то дополнительные действия, установленные модули и т.п. — всё это должно быть прописано в тексте в явном виде.

Сдача отчетов осуществляется через систему AnyTask.



# СЮДА надо теорию (если Амир не может, то мб Паша? Или я, но после среды тогда) нам нужно:
* Задание выполнили: Бакаров Амир, Никишина Ирина, Степачев Павел (группа МКЛ171)
* Краткую постановку задачи и формулировку задания
* Описание **минимума** необходимой теории и/или описание используемых инструментов - не стоит переписывать лекции или Википедию

Кластеризация новостей
Входная коллекция данных состоит из двух частей:
1. events.csv – список 28 резонансных событий первой половины 2017 года, каждому событию присвоен свой порядковый номер (id)
2. raw_news.csv – тексты новостей из различных новостных источников, известно, к какому резонансному событию относится каждая новость (столбец event_id).

Будем считать, что одно событие – это один кластер. В этом домашнем задании вам предстоит:
1. провести кластеризацию текстов новостей и проверить, получается ли восстановить кластерную структуру 
2. проверить, можно ли использовать кластерный анализ для обобщения: найти небольшое число кластеров и проверить, получается ли выделить общие направлени новостей

### Часть 1 [2 балла] Предварительная обработка текстов
Проведите предобработку новостей: токенизацию, приведение к нижнему регистру, лемматизацию. Проверьте, есть ли в коллекции дубликаты. Посчитайте, сколько новостей относится к каждому резонансному событию. 

Импорт необходимых библиотек:

In [21]:
# -*- coding: utf-8 -*-
from os import path
import glob
import pickle
import gensim
import numpy as np
import pickle
import time
from nltk.tokenize import RegexpTokenizer
from re import sub
from pandas import DataFrame
from pymystem3 import Mystem
import texterra
from pymorphy2 import MorphAnalyzer
import codecs
import csv
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer
from sklearn.metrics import *
from sklearn.cluster import *
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import *
import scipy.cluster.hierarchy as hac
import matplotlib.pyplot as plt

Было решено опробовать каждый из известных нам лемматизаторов (Mystem, pymorphy2 и Texterra) и понять, какой из них лучше работает для нашей задачи. 

Создаем экземпляры классов для каждого лемматизатора:

In [14]:
API_KEY = '9988cfb979b80264baeba1386cc7e455f99f943c'

morph = MorphAnalyzer()
m = Mystem()
t = texterra.API(API_KEY)
alpha_tokenizer = RegexpTokenizer('\w+')


Загрузим данные из файлов 'events.csv' и 'raw_news.csv':

In [65]:
df_events = DataFrame.from_csv('events.csv')
df_news = DataFrame.from_csv('raw_news.csv')
texts = list(df_news.text.values)

In [66]:
df_news.head()

,event_id,text
0,1,В ПЕТЕРБУРГЕ ПРОШЕЛ МИТИНГ ПРОТИВ ПЕРЕДАЧИ ИС...
1,1,"Lenta.co, Москва, 14 января 2017 СИТУАЦИЯ С П..."
2,1,"Аргументы и Факты (aif.ru), Москва, 14 января..."
3,1,"Google Новости ТОП, Москва, 14 января 2017 АК..."
4,1,"Газета.Ru, Москва, 13 января 2017 В МОСКОВСКО..."


Сколько новостей относится к каждому резонансному событию?

### Need graph?

In [67]:
df_news.event_id.value_counts()

17    102
28    100
18    100
7     100
10    100
12    100
27    100
16    100
1     100
25    100
26    100
21    100
23    100
3      84
22     82
9      82
24     62
4      62
2      51
11     49
19     45
6      41
8      27
13     24
20      8
15      7
5       2
14      2
Name: event_id, dtype: int64

Есть ли в коллекции дубликаты?

In [71]:
df_news[df_news.duplicated()]

,event_id,text
513,9,"Коммерсантъ. Новости информ. центра, Москва, ..."
518,9,"Коммерсантъ. Новости информ. центра, Москва, ..."
522,9,"Коммерсантъ. Новости информ. центра, Москва, ..."
526,9,"Коммерсантъ. Новости информ. центра, Москва, ..."
528,9,"Коммерсантъ. Новости информ. центра, Москва, ..."
536,9,"Коммерсантъ. Новости информ. центра, Москва, ..."


In [72]:
df_news.drop_duplicates()

,event_id,text
0,1,В ПЕТЕРБУРГЕ ПРОШЕЛ МИТИНГ ПРОТИВ ПЕРЕДАЧИ ИС...
1,1,"Lenta.co, Москва, 14 января 2017 СИТУАЦИЯ С П..."
2,1,"Аргументы и Факты (aif.ru), Москва, 14 января..."
3,1,"Google Новости ТОП, Москва, 14 января 2017 АК..."
4,1,"Газета.Ru, Москва, 13 января 2017 В МОСКОВСКО..."
5,1,"Top Real Estate (topre.ru), Москва, 13 января..."
6,1,"Findnews.ru, Москва, 13 января 2017 ПОЛИЦИЯ Н..."
7,1,"ПРАВДА.info (pravda.info), Москва, 13 января ..."
8,1,"Голос Америки (golos-ameriki.ru), Москва, 13 ..."
9,1,"Выбор Народа (vybor-naroda.org), Москва, 13 я..."


Проведите предобработку новостей: токенизацию, приведение к нижнему регистру, лемматизацию. Проверьте, есть ли в коллекции дубликаты. 

Лемматизация (и автоматические приведение к нижнему регистру). Мы решили рассмотреть три варианта лемматизации, поскольку не все способны одинаково хорошо справляться с контекстной омонимией (как, например, во фразе "Запотело стекло. Варенье стекло по краю банки."):
### Вы там что-то другое хотели вместо банок

In [16]:
def normalize_with_pymorphy(tokens):
    return [morph.parse(word)[0].normal_form for word in tokens]

def normalize_with_mystem(tokens):
    return ''.join(m.lemmatize(' '.join(tokens))).split()

def normalize_with_texterra(tokens):
    time.sleep(10)
    text = ' '.join(tokens)
    return [token[3] for token in list(t.lemmatization(text))[0]]

In [17]:
all_type_of_tokens = {}
tokenizers = {'pymorphy': normalize_with_pymorphy, 'texterra': normalize_with_texterra}

Из-за того, что какждый раз данные лемматизировать слишком долго (особенно с Texterra), мы их преобразуем в поток байтов
### Отсюда надо убрать фунцкии (или нет?)

In [ ]:
for name, tokenizer in tokenizers.items():
    all_texts = []
    for text in texts:
        print(texts.index(text))
        text = sub(r'http\S+', '', text)
        tokens = alpha_tokenizer.tokenize(text)
        tokens = tokenizer(tokens)
        all_texts.append(tokens)
    print(all_texts)
    with open('tokens_from_' + name + '.pickle', 'wb') as f:
        pickle.dump(all_texts, f)

## Мы попробовали всякие CountVectorizer, TF-IDF, но лучше всего отработали Word2Vec и Doc2Vec

In [19]:
#Do we need smth here?

Загружаем наши леммы в словарь типа data = {'texterra' : [ [lemma1, lemma2 ...], [], [] ... [] ],  'mystem': [ ], etc.}:

In [ ]:
data = {}
for filename in glob.iglob(path.join(path.dirname(__file__),'lemmas_from_*.pickle'), recursive=True):
    with open(filename, 'rb') as f:
        data_lemmas = pickle.load(f)
        data[path.splitext(path.basename(filename))[0]] = data_lemmas

#### Далее для каждого типа лемм создаем W2V модели, матрицу векторов для всех текстов, сохраняем ее в pickle:

Определяем длину векторов для каждого слова:

In [ ]:
num_of_features = 200

In [ ]:
for name_of_alg, list_of_docs in data.items():
    
    model = gensim.models.Word2Vec(list_of_docs, size=num_of_features, min_count=30, window=30)
    #model.save(name_of_alg.replace('lemmas','w2v_model2')+'.mdl')
    
    vectors_list = []
    
    for text_id in range(len(list_of_docs)):
        vector_for_each_text = []
        
        for word in list_of_docs[text_id]:
            try:
                featureVec = np.zeros(shape=(1, num_of_features), dtype='float32')
                featureVec = np.add(featureVec, model[word])
                vector_for_each_text.append(featureVec)
                
            except KeyError:
                pass

        first_vector = np.array(vector_for_each_text[0])
        for i in range(1, len(vector_for_each_text)):
            first_vector = np.add(first_vector, vector_for_each_text[i])
            
        resultVec = np.divide(first_vector, len(vector_for_each_text))
        vectors_list.append(resultVec)

    vectors_array = np.array(vectors_list[0])
    for i in range(1, len(vectors_list)):
        vectors_array = np.vstack((vectors_array,vectors_list[i]))

    with open(name_of_alg.replace('lemmas', 'vectors2')+ '.pickle', 'wb') as f:
            pickle.dump(np.matrix(vectors_array), f)

То же самое происходит с Doc2Vec:

In [ ]:
for name_of_alg, list_of_docs in data.items():
    
    #тут короче преобразования для того, чтобы doc2vec нормально кушал тексты 
    sentences = [gensim.models.doc2vec.TaggedDocument(words=list_of_docs[doc], tags=[u'text']) for doc 
                 in range(len(list_of_docs))]
    model = gensim.models.doc2vec.Doc2Vec(sentences, size=num_of_features, min_count=70, window=70)
    #model.save(name_of_alg.replace('lemmas', 'd2v_model') + '.mdl')
    model.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)


    vectors_list = []
    for text_id in range(len(list_of_docs)):
        vector_for_each_text = []
        for word in list_of_docs[text_id]:
            try:
                featureVec = np.zeros(shape=(1, num_of_features), dtype='float32')
                featureVec = np.add(featureVec, model[word])
                vector_for_each_text.append(featureVec)
            except KeyError:
                pass
        first = np.array(vector_for_each_text[0])

        for i in range(1, len(vector_for_each_text)):
            first = np.add(first, vector_for_each_text[i])
        resultVec = np.divide(first, len(vector_for_each_text))
        vectors_list.append(resultVec)

    vectors_array = np.array(vectors_list[0])

    for i in range(1, len(vectors_list)):
        vectors_array = np.vstack((vectors_array, vectors_list[i]))
        
    with open(name_of_alg.replace('lemmas', 'vectors_d2v') + '.pickle', 'wb') as f:
            pickle.dump(np.matrix(vectors_array), f)

А теперь самое приятное -- кластеризация

Надо написать, что брали и почему...не знаю, почему

In [22]:
list_of_alg = [KMeans, AgglomerativeClustering, SpectralClustering]

Для отрисовки Agglomerative clustering:

In [ ]:
def plotClusters(a):
    z = hac.linkage(a, method='ward')
    hac.dendrogram(z)
    plt.tight_layout()
    plt.show()

Берем лэйблы из данных 'raw_news.csv':

In [80]:
labels = list(df_news.event_id)

Здесь для каждого типа лемм применяем PipeLine:

In [ ]:
for filename in glob.iglob('vectors_d2v_from_*.pickle', recursive=True):
    with open(filename, 'rb') as f:
        data_model = pickle.load(f)
        
        for alg in list_of_alg:
            print(alg, filename.replace('vectors_d2v_from_', '').replace('.pickle',''))
            
            pipeline = Pipeline([('tfidf', TfidfTransformer()),
                                 ('svd', TruncatedSVD(n_components=150)),
                                 ('norm', Normalizer()),
                                 ('clust', alg(n_clusters=28))
                                ])
            pipeline.fit(data_model)
            
            explained_variance = pipeline.named_steps['svd'].explained_variance_ratio_.sum()
            print("Explained variance of the SVD step: {}%".format(int(explained_variance * 100)))
                        
            clust_labels = pipeline.named_steps['clust'].labels_

            print("Homogeneity:", homogeneity_score(labels, clust_labels))
            print("Completeness:", completeness_score(labels, clust_labels))
            print("V-measure",  v_measure_score(labels, clust_labels))
            print("Adjusted Rand-Index:",  adjusted_rand_score(labels, clust_labels))
            print(confusion_matrix(labels, clust_labels))
            print('==============')
        
        if alg == AgglomerativeClustering:
            plotClusters(data_model)

Вот тут вот сверху confusion matrix, если мы будем рисовать ее...

Что-то...